### 1 - Importing The Libraries

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.regnet import RegNetX002
from tensorflow.keras.applications.efficientnet import EfficientNetB0
import matplotlib.pyplot as plt

### 2 - Importing The Datasets

In [ ]:
print("Dataset is creating...")

from google.colab import drive
drive.mount('/content/drive')

#The path of the datasets
data_dir = '/content/drive/MyDrive/Education/MSc/GTSRB'
train_path = '/content/drive/MyDrive/Education/MSc/GTSRB/Train'
test_path = '/content/drive/MyDrive/Education/MSc/GTSRB/Test/'

folders = os.listdir(train_path)
samples_dict = {}

for folder in folders:
    images_in_folder = os.listdir(train_path + '/' + folder)
    samples_dict[folder] = len(images_in_folder)

image_data = []
image_labels = []
print("Done!")

### 3 - Converting The Images to 32x32 Size

In [ ]:
print("Images are converting...")
class_num = len(os.listdir(train_path))
for i in range(class_num):
    path = train_path +'/'+ str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((32, 32))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)
print("Done!")

### 4 - Creating The Array of The Data and Labels

In [ ]:
print("The Array of The Data and Labels are creating...")
image_data = np.array(image_data)
image_labels = np.array(image_labels)
print(image_data.shape, image_labels.shape)
print("Done!")

### 5 - Preparing The Dataset

In [ ]:
print("The Dataset is Preparing...")
#The dataset is split into training data and test data.
x_train, x_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42, shuffle=True)

print("Training and test dataset information (# of images, image dimensions): ")
print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test :", x_test.shape)
print("y_test :", y_test.shape)

#Dataset normalization
x_train = x_train/255.0
x_test = x_test/255.0

#Dataset One-Hot Encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)
print("Done!")

### 6 - Defining The Number of Epochs

In [ ]:
epoch_number=10

### 7 - Run The Corresponding Cell to Run Related Neural Networks.

#### a) KolNet Model

In [ ]:
print("The Model is Creating...")
#Create a Neural Network Model
model = Sequential()

#Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Dense Layer
model.add(Flatten())#Conversion of inputs into a single vector
model.add(Dense(64, activation='relu'))

#Output Layer
model.add(Dense(class_num, activation='softmax'))

#Compiling The Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Training
history=model.fit(x_train, y_train, epochs=epoch_number, validation_data=(x_test, y_test), batch_size=64)

#Measuring The Accuracy of The Model With The Test Data
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Done!")

#### b) EfficientNet Model

In [ ]:
model = EfficientNetB0(weights=None, input_shape=(32, 32, 3), classes=43)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=epoch_number, validation_data=(x_test, y_test))

#### c) ResNet Model

In [ ]:
model = ResNet50(weights=None, input_shape=(32, 32, 3), classes=43)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=epoch_number, validation_data=(x_test, y_test))

### 8 - Accuracy of The Model and Saving the Model

In [ ]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)
accuracy = np.sum(predictions==y_true)/len(y_true)
print("Accuracy:", accuracy)

file_name = input("Enter the name of the data to save: ")
model.save(file_name + '.h5')

### 9 - Visualizing the result

In [ ]:
epochs_range = range(epoch_number)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()